In [1]:

from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input 
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Flatten, Dropout, Activation, Input, Conv2D, MaxPooling2D
from keras.models import Model
from keras import Sequential
from sklearn.preprocessing import LabelBinarizer
import numpy as np
import pickle
import os

In [2]:
learning_rate = 0.001 #diminui o valor do learning rate pois estamos usando Adam como otimizador

## Treinamento Rede Inception

In [3]:
def get_images(pasta):
    exts = ['.PNG','.JPG','.JPEG','.TIFF','.GIF','.BMP']
    tot_images = 0
    for sf in [name for name in os.listdir(pasta) if os.path.isdir(os.path.join(pasta, name))]:
        subdir = os.path.join(pasta,sf)
        tot_images = tot_images + len([name for name in os.listdir(subdir) if os.path.splitext(name)[1].upper() in exts])
    return tot_images

In [4]:
# FUNÇÃO PARA LER O NOME DAS CLASSES
def get_labels(pasta):
    return [name for name in os.listdir(pasta) if os.path.isdir(os.path.join(pasta, name))];


In [5]:
EPOCHS = 50;
BATCH_SIZE = 16; #serão gerados batches de 16 imagens (a rede vê 16 imagens de cada vez durante o treino). Esse número é para facilitar o processamento conforme a memória do computador durante o treino, colocamos 16 não travar
IMG_SIZE = (299,299,3);
optimizer_name = 'SGD'

In [6]:
trainFolder = 'C:\\Users\\LDT\\Desktop\\mestrado-unifesp\\db\\train_v2'
valFolder = 'C:\\Users\\LDT\\Desktop\\mestrado-unifesp\\db\\val_v2'
labels = get_labels(trainFolder)
labels = np.array(labels);

# Organiza os labels em matriz e salva, para posterior uso em classificação
lb = LabelBinarizer();
labels = lb.fit_transform(labels);
f = open('C:\\Users\\LDT\\Desktop\\mestrado-unifesp\\budioes_inception_v2_' + optimizer_name + '_' + str(learning_rate) + ".pickle", "wb")
f.write(pickle.dumps(lb));
f.close();

ImageDataGenerator = Carrega aos poucos as imagens em memória para fazer a leitura (para não estourar a memória)

In [7]:
#ImageDataGenerator = objeto para buscar as imagens em uma pasta
#Treino
#Quando carregar as imagens, será aplicada a função "preprocess_input" pré definida pela rede inception
#O "preprocess_input" modifica os valores da imagem para facilitar o processamento.
augTrain = ImageDataGenerator(preprocessing_function=preprocess_input, rotation_range=20, width_shift_range = 0.1, height_shift_range = 0.1, 
                              shear_range = 0.15, zoom_range = [1.0, 1.25], horizontal_flip=True, 
                              fill_mode="nearest");
#Validação
augVal = ImageDataGenerator(preprocessing_function=preprocess_input);

In [8]:
#Declaração da rede INCEPTION V3
# Pega só as camadas de convolução e retreina as de classificação

baseModel = InceptionV3(include_top=False, weights="imagenet", input_shape=(299,299,3), pooling='avg') 
#sempre usa 3 como parametro do input pois corresponde ao tamanho do RGB da imagem

In [9]:
#Como vamos usar imagenet, não faz sentido treinar a rede novamente pois já vamos usar o modelo treinado
for layer in baseModel.layers:
    layer.trainable = False

In [10]:
#Include_Top feito "Na mão":

#É preciso criar novas camadas (headModel) pra zerar os pesos da VGG
headModel = baseModel.output
headModel = Dense(len(lb.classes_), activation="softmax", name='predictions')(headModel)
#Junta tudo num modelo só
model = Model(inputs=baseModel.input, outputs=headModel)

In [11]:
#model.summary()

In [12]:
if optimizer_name == 'Adam':
    optimizer = Adam(learning_rate=learning_rate)

if optimizer_name == 'SGD':
    optimizer = SGD(learning_rate=learning_rate)

In [13]:
#No nosso modelo, o callback irá salvar o melhor modelo entre as épocas (epochs) - função Model Checkpoint
callbacks = [
    #ReduceLROnPlateau(monitor = 'val_acc',factor=0.85, patience=10, min_lr=0.000001, verbose=1),
    ModelCheckpoint('C:\\Users\\LDT\\Desktop\\mestrado-unifesp\\exemplos\\modelo_budioes_inception_v2_' + optimizer_name + '_' + str(learning_rate) + 
                    '-ckpnt.model', save_best_only=True, monitor='val_accuracy', mode='max', verbose = 1)
]  

In [14]:
#Reservando espaço de memória para a rede funcionar
#Ao final de cada época, será rodado o comando "callbacks"
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])

In [15]:
#Função para buscar imagens
#TREINO
trainGen = augTrain.flow_from_directory(
    trainFolder, #caminho da imagem
    class_mode="categorical", #o nome da pasta onde está a imagem será o nome da classe
    target_size=(IMG_SIZE[0], IMG_SIZE[1]), #tamanho da imagem a ser redimensionada
    color_mode="rgb", #a imagem terá 3 canais RGB
    shuffle=True, #vai embaralhar as imagens enquanto faz a leitura
    batch_size=BATCH_SIZE); #de quantas em quantas imagens será feita a leitura (tamanho do BATCH)

Found 594 images belonging to 3 classes.


In [16]:
#Função para buscar imagens
#VALIDAÇÃO
valGen = augVal.flow_from_directory(
    valFolder,
    class_mode="categorical",
    target_size=(IMG_SIZE[0], IMG_SIZE[1]),
    color_mode="rgb",
    shuffle=True,
    batch_size=BATCH_SIZE);

Found 122 images belonging to 3 classes.


In [17]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 149, 149, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 149, 149, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [18]:
#fit_generator vai de fato treinar a rede
trained_model = model.fit(trainGen, validation_data=valGen,
                        steps_per_epoch=get_images(trainFolder)//BATCH_SIZE,
                        validation_steps=get_images(valFolder) // BATCH_SIZE,
                        epochs = EPOCHS, callbacks=callbacks, verbose =1);
#Dividir o número de imagens pelo número de batchs para garantir que cada BATCH seja lido a cada época
#tanto no treino (steps_per_epoch) quanto na validação (validation_steps)
#verbose = dá output da rede a cada final de época


Epoch 1/50
37/37 [==============================] - 27s 480ms/step - loss: 1.1510 - accuracy: 0.3183 - val_loss: 1.1073 - val_accuracy: 0.3750

Epoch 00001: val_accuracy improved from -inf to 0.37500, saving model to C:\Users\LDT\Desktop\mestrado-unifesp\exemplos\modelo_budioes_inception_v2_SGD_0.001-ckpnt.model
INFO:tensorflow:Assets written to: C:\Users\LDT\Desktop\mestrado-unifesp\exemplos\modelo_budioes_inception_v2_SGD_0.001-ckpnt.model\assets
Epoch 2/50
37/37 [==============================] - 14s 375ms/step - loss: 1.0661 - accuracy: 0.4172 - val_loss: 1.0259 - val_accuracy: 0.4821

Epoch 00002: val_accuracy improved from 0.37500 to 0.48214, saving model to C:\Users\LDT\Desktop\mestrado-unifesp\exemplos\modelo_budioes_inception_v2_SGD_0.001-ckpnt.model
INFO:tensorflow:Assets written to: C:\Users\LDT\Desktop\mestrado-unifesp\exemplos\modelo_budioes_inception_v2_SGD_0.001-ckpnt.model\assets
Epoch 3/50
37/37 [==============================] - 14s 365ms/step - loss: 0.9932 - accurac

In [19]:
#Salva o modelo (pesos + conexões entre os neurônios, ou seja, a estrutura da rede)
model.save('C:\\Users\\LDT\\Desktop\\mestrado-unifesp\\exemplos\\modelo_budioes_inception_v2_' + optimizer_name + '_' + str(learning_rate) + ".model");

INFO:tensorflow:Assets written to: C:\Users\LDT\Desktop\mestrado-unifesp\exemplos\modelo_budioes_inception_v2_SGD_0.001.model\assets


In [20]:
#https://www.learndatasci.com/tutorials/hands-on-transfer-learning-keras/
#https://neptune.ai/blog/keras-metrics